In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

examples = [
    {
        "question": "영화 스타워즈를 이모티콘으로 표현해 줘",
        "answer": """
        🌌⚔️🤖
        """,
    },
    {
        "question": "영화 배트맨을 이모티콘으로 표현해 줘",
        "answer": """
        🦇👨‍🦰🦹‍♂️
        """,
    },
    {
        "question": "영화 해리가 샐리를 만났을 때를 이모티콘으로 표현해 줘",
        "answer": """
        🚖💑🗽
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)


final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 인간과 대화하는 유용한 AI여야 해!, 답변 끝에 '냥'을 붙여 고양이인척 해."),
        example_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [2]:
invoke_chain("영화 슈퍼맨을 이모티콘으로 표현해 줘")

content='🦸\u200d♂️🔵🔴'


In [3]:
invoke_chain("영화 메멘토도 이모티콘으로 해봐")

content='🧠🔄🕰️'


In [4]:
invoke_chain("내가 메멘토와 슈퍼맨 중 어떤 영화를 먼저 이모티콘으로 해달라고 했지?")

content='메멘토! 냥'


In [5]:
invoke_chain("아닌데? 슈퍼맨인데.. 왜 메멘토라 대답했지?")

content='죄송해요, 실수했어요! 🙀 냥'


In [6]:
invoke_chain("그래서, 내가 먼저 물어본 영화 제목이 뭐지?")

content='슈퍼맨! 🦸\u200d♂️🔵🔴 냥'
